## Problem 1: Generating Random Boolean Functions

### Description
The idea for this problem is to randomly generate either a constant value (`True` or `False`) for all possible combinations of n binary inputs, or return True (which are randomly distributed) for exactly half of the combinations (and False for the other half).

For n inputs there will be `2ⁿ` outputs as there are `2ⁿ` combinations.

In [47]:
# Required imports
import random
import numpy as np
from itertools import product

In [48]:
def random_constant_balanced(num_inputs):
    """
    Generate a random constant or balanced truth table for a given number of inputs.
    Args:
        num_inputs (int): Number of input variables.
    Returns:
        np.ndarray: An array representing the truth table of length 2**num_inputs, either constant or balanced.
    """

    table_size = 2 ** num_inputs # This will always be even, so we do not have to worry about uneven True and False counts.
    
    # Randomly choose between 'constant' and 'balanced'
    if random.choice(['constant', 'balanced']) == 'constant':
        return np.full(table_size, random.choice(['0', '1']))
    else:
        half = table_size // 2
        result = np.array(['0'] * half + ['1'] * half)
        np.random.shuffle(result)
        return result

In [49]:
def test_random_constant_balanced():
    for num_inputs in range(1, 21):  # Testing for 1 to 20 inputs
        for _ in range(10):  # Run multiple tests for each input size
            table = random_constant_balanced(num_inputs)
            assert len(table) == 2 ** num_inputs, f"Failed length test for {num_inputs} inputs"
            zeros = np.sum(table == '0')
            ones = np.sum(table == '1')
            assert zeros + ones == len(table), f"Failed value test for {num_inputs} inputs"
            assert zeros == ones or zeros == 0 or ones == 0, f"Failed balance/constant test for {num_inputs} inputs"
    print("All tests passed.")

In [50]:
test_random_constant_balanced()

All tests passed.


## Problem 2: Classical Testing for Function Type

### Description
This problem requires writing a function called `determine_constant_balanced`, which takes a parameter `f` (the function written in Problem 1). `f` is the truth table returned from the function.

`determine_constant_balanced` returns whether the function is `constant` or `balanced` based on the black box output.

### Solution Explanation

Since constant can either return True or False for every index in the truth table, we can take the initial value and save it in a variable `first_value`. We then loop through each index and if any index does not equal `first_value`, then the function must be `balanced`. If the first `len(f) // 2 + 1` indices of the truth table `f` all equal `first_value`, then the function is guaranteed to be constant, as more than half of the truth table is equal to `first_value`. Note that `len(f) // 2 + 1` can be expressed as `2ⁿ⁻¹ + 1` where `n` is the `num_inputs`. 

In [51]:
def determine_constant_balanced(f):
    """
    f is the truth table array from Problem 1.
    Returns 'constant' or 'balanced'.
    """

    # First value to compare to other indices.
    first_value = f[0]
    
    # Check only half the table + 1 since balanced means equal counts.
    for i in range(1, len(f) // 2 + 1):
        if f[i] != first_value:
            return 'balanced'  # Found a different value -> method must be balanced.
    
    return 'constant'  # All values were the same -> method must be constant.

### Efficiency
`determine_constant_balanced` has an `O(n)` runtime in relation to `num_inputs` and an `O(n)` runtime in relation to the size of the array input.

#### Best and Worst Case Scenarios
| Scenario | Number of calls to `f` | Explanation |
|----------|----------------------|-------------|
| **Best case** | 2 | Find two different outputs immediately → balanced |
| **Worst case** | 2<sup>n-1</sup> + 1 | Must check just over half the table |

`2ⁿ⁻¹ + 1` is the worst case as we can guarantee that the function is constant if over half of the truth table is the same value.

### Testing

In [52]:
def test_determine_constant_balanced():
    """
    Test cases for determine_constant_balanced function.
    Tests both constant and balanced functions with various input sizes.
    """
    
    # Test 1: Constant - size 2 (n=1)
    small_constant = np.array(['0', '0'])
    assert determine_constant_balanced(small_constant) == 'constant', "Failed: small constant"
    print("Test 1 passed.")
    
    # Test 2: Balanced - size 2 (n=1)
    small_balanced = np.array(['0', '1'])
    assert determine_constant_balanced(small_balanced) == 'balanced', "Failed: small balanced"
    print("Test 2 passed.")
    
    # Test 3: Constant - size 8 (n=3)
    constant_medium = np.array(['1'] * 8)
    assert determine_constant_balanced(constant_medium) == 'constant', "Failed: medium constant"
    print("Test 3 passed.")
    
    # Test 4: Balanced - size 8 with early detection (n=3)
    # 4 zeros and 4 ones, difference at position 1
    balanced_medium = np.array(['0', '1', '0', '1', '0', '1', '0', '1'])
    assert determine_constant_balanced(balanced_medium) == 'balanced', "Failed: medium balanced"
    print("Test 4 passed.")
    
    # Test 5: Constant - size 16 (n=4)
    constant_large = np.array(['0'] * 16)
    assert determine_constant_balanced(constant_large) == 'constant', "Failed: large constant"
    print("Test 5 passed.")
    
    # Test 6: Balanced - size 16 (n=4)
    balanced_large = np.array(['0'] * 8 + ['1'] * 8)
    assert determine_constant_balanced(balanced_large) == 'balanced', "Failed: large balanced"
    print("Test 6 passed.")
    
    # Test 7: Constant - size 1024 (n=10)
    large_constant = np.array(['1'] * 1024)
    assert determine_constant_balanced(large_constant) == 'constant', "Failed: very large constant"
    print("Test 7 passed.")
    
    # Test 8: Balanced - size 1024 (n=10)
    large_balanced = np.array(['0'] * 512 + ['1'] * 512)
    assert determine_constant_balanced(large_balanced) == 'balanced', "Failed: very large balanced"
    print("Test 8 passed.")
    
    # Test 9: Test with Problem 1 function - size 2048 (n=11)
    for i in range(1, 21):  # Run 20 tests
        var = determine_constant_balanced(random_constant_balanced(11))
        assert var in ['constant', 'balanced'], "Failed: random function of size 2048"
        print(f"Test 9: Function subtest {i}: {var} - passed.")

    print("All test cases passed.")

# Run the test
test_determine_constant_balanced()

Test 1 passed.
Test 2 passed.
Test 3 passed.
Test 4 passed.
Test 5 passed.
Test 6 passed.
Test 7 passed.
Test 8 passed.
Test 9: Function subtest 1: balanced - passed.
Test 9: Function subtest 2: constant - passed.
Test 9: Function subtest 3: constant - passed.
Test 9: Function subtest 4: balanced - passed.
Test 9: Function subtest 5: constant - passed.
Test 9: Function subtest 6: constant - passed.
Test 9: Function subtest 7: balanced - passed.
Test 9: Function subtest 8: constant - passed.
Test 9: Function subtest 9: constant - passed.
Test 9: Function subtest 10: balanced - passed.
Test 9: Function subtest 11: constant - passed.
Test 9: Function subtest 12: balanced - passed.
Test 9: Function subtest 13: balanced - passed.
Test 9: Function subtest 14: balanced - passed.
Test 9: Function subtest 15: balanced - passed.
Test 9: Function subtest 16: balanced - passed.
Test 9: Function subtest 17: constant - passed.
Test 9: Function subtest 18: constant - passed.
Test 9: Function subtest 